# Config

In [1]:
import os
import cv2
import numpy as np 
import random
import numpy as np
from scipy.ndimage.filters import generic_filter

# Input Image Dir
image_dir = "train/images/"
image_target_dir = "train/targets/"

# Output Dir
output_image_dir = "blurred_images/train/images/"
output_target_dir = "blurred_images/train/targets/"

# Noise Parameters 

gaussian_stddev = 0.35
salt_pepper_probability = 0.002

# Majority Filter Parameters 

perform_majority_filtering = 1
neighborhood_size = (3, 3)  # 3x3 neighborhood
threshold = 8  # Threshold value

/var/folders/3y/1js5b25s1b1dk8g7drn7_g8h0000gn/T/ipykernel_9426/423149139.py:6: DeprecationWarning: Please use `generic_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import generic_filter


## Visualize Noisy Images

In [2]:
# List all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith((".jpg", ".jpeg", ".png"))]

indx = random.randint(0, len(image_files))
image_file = image_files[indx]


# Read the image
image_path = os.path.join(image_dir, image_file)
image = cv2.imread(image_path)

# Add Gaussian noise
mean = 0
stddev = gaussian_stddev
gaussian_noise = np.random.normal(mean, stddev, image.shape).astype(np.uint8)
noisy_image_gaussian = cv2.add(image, gaussian_noise)

# Add salt and pepper noise
salt_pepper_noise = np.zeros(image.shape, np.uint8)
probability = salt_pepper_probability
salt = np.where(np.random.rand(*image.shape[:2]) < probability)
pepper = np.where(np.random.rand(*image.shape[:2]) < probability)
salt_pepper_noise[salt] = 255
salt_pepper_noise[pepper] = 0
noisy_image_salt_pepper = cv2.add(image, salt_pepper_noise)

cv2.imshow("Original Image", image)
cv2.imshow("Noisy Image (Salt&Pepper)", noisy_image_salt_pepper)
cv2.imshow("Noisy Image (Gaussian)", noisy_image_gaussian)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)


-1

In [3]:
def majority_filter(mask, neighborhood_size, threshold):
    def majority_value(arr):
        unique_classes, counts = np.unique(arr, return_counts=True, axis=0)
        max_count = np.max(counts)
        if max_count >= threshold:  # Check if the max count crosses the threshold
            majority_class = unique_classes[np.argmax(counts)]
            return majority_class
        else:
            return arr[int(len(arr)/2)]  # Keep the original value

    filtered_mask = np.zeros_like(mask)

    for channel in range(mask.shape[2]):
        channel_filtered = generic_filter(mask[:, :, channel], majority_value, size=neighborhood_size)
        filtered_mask[:, :, channel] = channel_filtered

    return filtered_mask

def generate_noisy_images(input_dir,image_target_dir,
                          output_image_dir,output_target_dir,
                          gaussian_stddev,salt_pepper_probability,
                         perform_majority_filtering,
                         early_stopping = False):

    # Create separate folders for different types of noise
    output_dir_image = os.path.join('', output_image_dir)
    output_dir_mask = os.path.join('', output_target_dir)


    os.makedirs(output_dir_image, exist_ok=True)
    os.makedirs(output_dir_mask, exist_ok=True)


    # List all image files in the directory
    image_files = [f for f in os.listdir(input_dir) if f.endswith((".jpg", ".jpeg", ".png"))]

    for image_file in image_files:
        # Read the image
        image_path = os.path.join(image_dir, image_file)
        image = cv2.imread(image_path)
        
        # Read Mask
        image_mask_path = os.path.join(image_target_dir, image_file[:-4] + '_target.png')
        mask = cv2.imread(image_mask_path)        
        
        if perform_majority_filtering:
            mask = majority_filter(mask, neighborhood_size, threshold)
        

        # Add Gaussian noise
        mean = 0
        stddev = gaussian_stddev
        gaussian_noise = np.random.normal(mean, stddev, image.shape).astype(np.uint8)
        noisy_image_gaussian = cv2.add(image, gaussian_noise)

        # Add salt and pepper noise
        salt_pepper_noise = np.zeros(image.shape, np.uint8)
        probability = salt_pepper_probability
        salt = np.where(np.random.rand(*image.shape[:2]) < probability)
        pepper = np.where(np.random.rand(*image.shape[:2]) < probability)
        salt_pepper_noise[salt] = 255
        salt_pepper_noise[pepper] = 0
        noisy_image_salt_pepper = cv2.add(image, salt_pepper_noise)



        # Save the noisy images in separate folders
        output_path_gaussian = os.path.join(output_dir_image, image_file[:-4]+'_gaussian'+image_file[-4:])
        output_path_salt_pepper = os.path.join(output_dir_image, image_file[:-4]+'_salt_pepper'+image_file[-4:])
        output_path_original = os.path.join(output_dir_image, image_file)
        
        # Mask for different Noise
        output_path_gaussian_mask = os.path.join(output_dir_mask, image_file[:-4]+'_target_gaussian'+image_file[-4:])
        output_path_salt_pepper_mask = os.path.join(output_dir_mask, image_file[:-4]+'_target_salt_pepper'+image_file[-4:])
        output_path_original_mask = os.path.join(output_dir_mask, image_file[:-4]+'_target'+image_file[-4:])
        
        # Saving
        cv2.imwrite(output_path_gaussian, noisy_image_gaussian)
        cv2.imwrite(output_path_salt_pepper, noisy_image_salt_pepper)
        cv2.imwrite(output_path_original, image)
        
        cv2.imwrite(output_path_gaussian_mask,mask)
        cv2.imwrite(output_path_salt_pepper_mask,mask)
        cv2.imwrite(output_path_original_mask,mask)    
        
        if early_stopping:
            break

    print("Noisy images saved.")

# def gen_noisy_image_mask(image_target_dir,output_target_dir):

#     output_dir = os.path.join('', output_target_dir)
#     os.makedirs(output_dir, exist_ok=True)

#     image_files = [f for f in os.listdir(image_target_dir) if f.endswith((".jpg", ".jpeg", ".png"))]

#     for image_file in image_files:
#         # Read the image
#         image_path = os.path.join(image_target_dir, image_file)
#         image = cv2.imread(image_path)

#         # Save the noisy images in separate folders
#         output_path_gaussian = os.path.join(output_dir, image_file[:-4]+'_gaussian'+image_file[-4:])
#         output_path_salt_pepper = os.path.join(output_dir, image_file[:-4]+'_salt_pepper'+image_file[-4:])
      

#         cv2.imwrite(output_path_gaussian, image)
#         cv2.imwrite(output_path_salt_pepper, image)
        
#         break


#     print("Noisy Image masked Saved")
    

In [ ]:
generate_noisy_images(image_dir,image_target_dir,
                      output_image_dir,output_target_dir,
                      gaussian_stddev,salt_pepper_probability,
                     perform_majority_filtering = 1 ,
                     early_stopping = True)


In [ ]:
# Bin the Mask - ( 0 - No Damage , 1 Damaged )

In [ ]:
pre_image_path = 'blurred_images/train/images/hurricane-harvey_00000015_pre_disaster_salt_pepper.png'
target_path = 'blurred_images/train/targets/hurricane-harvey_00000015_pre_disaster_target_salt_pepper.png'
original_target_path = 'train/targets/hurricane-harvey_00000015_pre_disaster_target.png'

pre_image = cv2.imread(pre_image_path)
target = cv2.imread(target_path, cv2.IMREAD_GRAYSCALE)

print("The Image Shape : {} and target Shape : {}".format(pre_image.shape,target.shape))

original_segmented_image = cv2.bitwise_and(pre_image, pre_image, mask=target)

print("The shape of Original segmented Image :{}".format(original_segmented_image.shape))


# Display the images and target mask for visualization (optional)
cv2.imshow('Pre-disaster Image', pre_image)
cv2.imshow('Target Mask', target)
cv2.imshow('Filtered Segmented Image', original_segmented_image)
cv2.imshow('Original Segmented Image', cv2.bitwise_and(pre_image, pre_image, 
                                                       mask=cv2.imread(original_target_path, cv2.IMREAD_GRAYSCALE)))

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)